In [28]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from selenium import webdriver

In [29]:
executable_path = {'executable_path':'C:/Users/woodf/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [11]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
html = browser.html
soup = bs(html, 'lxml')

In [12]:
#get first title
results = soup.find('div', class_='content_title').text
news_title = results.strip()
news_title


"NASA's Mars 2020 Heads Into the Test Chamber"

In [13]:
#get first p
news_p = soup.find('div', class_="article_teaser_body").text
news_p

'In this time-lapse video taken at JPL, engineers move the Mars 2020 rover into a large vacuum chamber for testing in Mars-like environmental conditions.'

In [14]:
#load spaceimages into splinter and soup
url_pic = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_pic)
html2 = browser.html
soup2 = bs(html2, 'html.parser')

In [15]:
#clicking button element via splinter isn't working so will "click" via soup 
button = soup2.find('a', class_="button fancybox")
button_click = "https://www.jpl.nasa.gov" + button["data-link"]
button_click


'https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA19330'

In [16]:
btcl = soup2.find('a', class_="button fancybox")["data-link"]
btcl

'/spaceimages/details.php?id=PIA19330'

In [17]:
#use splinter to navigate to link
browser.visit(button_click)
html3=browser.html
soup3 = bs(html3, 'html.parser')

In [18]:
#get largesize image link
large = soup3.find('figure', class_='lede')
featured_image_url = "https://www.jpl.nasa.gov" + large.a['href']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19330_hires.jpg'

In [19]:
#mars weather, open site in splinter and create soup object
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)
html4 = browser.html
soup4 = bs(html4, 'html.parser')

In [20]:
#find first text paragraph and delete anchor tet
link = soup4.find('div', class_='js-tweet-text-container')
mars = link.find('a')
mars.decompose()


In [21]:
#assign variable to weather text 
mars_weather = link.p.text
mars_weather

'InSight sol 338 (2019-11-08) low -102.5ºC (-152.5ºF) high -24.0ºC (-11.1ºF)\nwinds from the SSE at 5.3 m/s (11.9 mph) gusting to 19.6 m/s (43.9 mph)\npressure at 6.90 hPa'

In [22]:
#get table from space facts
facts_url = "https://space-facts.com/mars/"
facts_table = pd.read_html(facts_url)

In [39]:
#create df
table_df = facts_table[1]

In [24]:
#convert to HMTL string and clean it
html_table = table_df.to_html(index=False)
html_table = html_table.replace('\n','')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Mars - Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <td>Temperature:</td>      <td>-153 to 20 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [37]:
type(html_table)

str

In [25]:
#write to text
text_file = open("planet_table.html", "w")
text_file.write(html_table)
text_file.close()

In [30]:
#hemispheres
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
hemi_html = browser.html
hemi_soup = bs(hemi_html, 'html.parser')

In [31]:
items = hemi_soup.find_all('div', class_='item')

In [33]:
#once again, clicking button element via splinter isn't working so will "click" via soup 
#loop through each page, create dictionary for each page and append to a hemisphere list

hemi_list = []
for item in items:
    link = item.find('a', class_ = "itemLink product-item")
    click = "https://astrogeology.usgs.gov" + link["href"]
    browser.visit(click)
    html_click = browser.html
    soup_click = bs(html_click, 'html.parser')
    title = soup_click.find('h2', class_ = "title").text.rstrip("Enhanced")
    href = soup_click.find('div', class_ = "downloads").find("li").find('a')["href"]
    click_dict = {}
    click_dict["title"] = title
    click_dict["img_url"] = href
    hemi_list.append(click_dict)

    
    

In [34]:
#check list
hemi_list

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [36]:
hemi_list[0]['title']

'Cerberus Hemisphere '

In [331]:
#write this file to python script
!jupyter nbconvert --to script mission_to_mars.ipynb

[NbConvertApp] Converting notebook mission_to_mars.ipynb to script
[NbConvertApp] Writing 4164 bytes to mission_to_mars.py


In [40]:

    mars_data = {"latest_headline":news_title, "latest_news": news_p, "featured_image": featured_image_url,
                "weather": mars_weather, "facts_table": html_table, "hemispheres": hemi_list}

In [47]:
mars_data["hemispheres"][2]["img_url"]

'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'